In [ ]:
import pandas as pd
import keras
import sklearn # pip install scikit-learn==1.5.0
import scikeras

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras import Sequential
from keras import backend as k
from sklearn.model_selection import GridSearchCV # Essa classe é utilizada para se fazer a combinação de vários parâmetros dentro da rede neural, nos permitindo descobrir qual os melhores parâmetros de forma automática. Ela ainda faz a validação cruzada

In [ ]:
"""Atributos previsores"""
x = pd.read_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/x.csv')
x.drop(['id', 'diagnosis'], axis='columns', inplace=True)

"""Arquivo binário"""
y = pd.read_csv('/home/vitor-lima/Documents/jupyter/deep_learning_com_python_de_a_a_z/breast_cancer_wisconsin_diagnostic/y.csv')

In [ ]:
"""Para trabalhar com validação cruzada, precisamos de utilizar uma função para criar a rede neural"""
def network_creation(optimizer, loss, kernel_initializer, activation, neurons):
    k.clear_session() # Essa biblioteca é o backend do tensorflow. É importante limpar a sessão antes de criarmos a estrutura da rede neural
    
    neural_net = Sequential([ # Mesma rede que criamos antes
        keras.layers.InputLayer(shape=(10,)),
        keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        keras.layers.Dropout(rate=0.2), # Dropout na primeira camada oculta
        keras.layers.Dense(units=6, activation=activation, kernel_initializer=kernel_initializer),
         keras.layers.Dropout(rate=0.2), # Dropout na segunda camada oculta
        keras.layers.Dense(units=1, activation='sigmoid')
    ])

    neural_net.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])

    return neural_net

In [ ]:
"""Criando a estrutura da rede neural"""
neural_net = KerasClassifier(model=network_creation)

In [ ]:
parameters = { # Variável utilizada para testar diferentes configurações de parâmetros
    'batch_size': [10, 30],
    'epochs': [50],
    'model__optimizer': ['adam'],
    'model__loss': ['binary_crossentropy'], # hinge é uma outra função que pode ser utilizada em problemas de classificação binária
    'model__kernel_initializer': ['random_uniform'],
    'model__activation': ['relu'],
    'model__neurons': [16],
}

# O GridSearchCV vai fazer todas as combinações possíveis com as opções acima para encontrar a melhor. Para encontrar quantas combinações serão testada, basta fazer a multiplicação entre as quantidades de opções de cada parâmetro. Nesse caso 2^7 = 128

In [ ]:
grid_search = GridSearchCV(estimator=neural_net, param_grid=parameters, scoring='accuracy', cv=5) # Definimos igual a 5 apenas para não demorar muito, pois esse valor multiplicará o 2^7 acima

In [ ]:
grid_search = grid_search.fit(X=x, y=y)

In [ ]:
best_parameters = grid_search.best_params_
best_parameters

In [ ]:
best_precision = grid_search.best_score_
best_precision